In [ ]:
!# Download YOLOv7 code
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!ls

In [ ]:
#count and generate files

import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
import argparse
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel


gts = []
detections = []

device = "cpu"
imgsz=640
weights = "./runs/train/yolov7x/weights/best.pt"
model = attempt_load(weights, map_location=device)
# model.eval()
stride = int(model.stride.max())  # model stride
imgsz = check_img_size(imgsz, s=stride)
half = False
names = model.module.names if hasattr(model, 'module') else model.names


img_path = "../train_test_dataset/test/images" #yolo dataset images location, 
dataset = LoadImages(img_path,imgsz,stride)

conf_thres = 0.5
iou_thres = 0.8
colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

for path, img, im0s, vid_cap in tqdm(dataset):
  img = torch.from_numpy(img).to(device)
  img = img.half() if half else img.float()  # uint8 to fp16/32
  img /= 255.0  # 0 - 255 to 0.0 - 1.0
  label_path = path.replace("images",'labels')[:-3]+"txt"
  with open(label_path,'r') as fp:
    lines = fp.readlines()
    gts.append({'file':path,'gts':[names[int(line[0])] for line in lines]})
  if img.ndimension() == 3:
    img = img.unsqueeze(0)
  t1 = time_synchronized()
  with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
    pred = model(img)[0]
  im0 = im0s
  s=''
  # Apply NMS
  pred = non_max_suppression(pred, conf_thres, iou_thres)
  detection=[]
  for i, det in enumerate(pred):  # detections per image
    
    gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
    
    if len(det):
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

        for c in det[:, -1].unique():
            n = (det[:, -1] == c).sum()  # detections per class
            s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
            detection+= [names[int(c)]]

        for *xyxy, conf, cls in reversed(det):
            label = f'{names[int(cls)]} {conf:.2f}'
            plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=2)

  # print(s)  
  detections.append({'file':path,'detections':detection})



import pandas as pd
def count_class(data,name):
  count = 0
  for n in data:
    if n == name:
      count+=1
  return count

det_df = pd.DataFrame(detections)
gt_df = pd.DataFrame(gts)
merged_df = pd.merge(gt_df,det_df,on='file')
# merged_df.head()
for name in names:
  merged_df['det_'+name] = merged_df['detections'].apply(lambda x: count_class(x,name) )
  merged_df['gt_'+name] = merged_df['gts'].apply(lambda x: count_class(x,name) )
merged_df
# merged_df.to_csv("results.csv")

              